<a href="https://colab.research.google.com/github/asegura4488/MetodosComputacionalesI2025/blob/main/Semana10/MinimosGradienteLevenberg%E2%80%93Marquardt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/ColabNotebooks/Cursos/MetodosComputacionalesI2025/Semana10/')
!ls

Data  MinimosGradiente.ipynb  MinimosGradienteLevenberg–Marquardt.ipynb


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import sympy as sym
import time

In [ ]:
data = np.loadtxt('Data/MinimosLineal.txt')
x_data, y_data = data[:, 0], data[:, 1]

In [ ]:
def Plotter(e):

  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(221)
  ax1 = fig.add_subplot(222)
  ax2 = fig.add_subplot(223)

  ax.set_title('Epoch: {:.0f}'.format(e),fontsize=10)

  return fig,ax,ax1,ax2

In [ ]:
def Model(p,x):

  y = 0.
  for i in range(0,len(p)):
    y += p[i]*x**i

  return y

In [ ]:
_x  = sym.Symbol('x',real=True)
p = np.array([4,3.])
Model(p,_x)

3.0*x + 4.0

In [ ]:
Model(p,x_data)

array([ 4.      ,  7.157896, 10.315789, 13.473685, 16.631578, 19.789474,
       22.947367, 26.105263, 29.263159, 32.421052, 35.57896 , 38.73685 ,
       41.89474 , 45.05263 , 48.21052 , 51.36841 , 54.52633 , 57.68422 ,
       60.84211 , 64.      ])

In [ ]:
# Generamos el vector
def GetF(p, tipo="lineal"):
    return np.array([ y_data[i] - Model(p, x_data[i]) for i in range(len(x_data)) ])

In [ ]:
G = GetF(p)
G

array([ -8.786585 , -11.223766 , -10.1361396, -10.753878 , -13.753055 ,
       -15.262048 , -14.119633 , -15.128043 , -17.463719 , -17.891742 ,
       -18.35433  , -17.63875  , -21.10365  , -18.97245  , -22.2075   ,
       -21.22308  , -25.38386  , -24.39988  , -26.33687  , -27.43738  ])

In [ ]:
def Jacobiano(p, x_data):

    n = len(p)
    N = len(x_data)
    J = np.zeros((N, n))

    for j in range(n):
        J[:, j] = -x_data**j  # primer columna: -1, luego -x, luego -x^2, etc.
    return J

In [ ]:
J = Jacobiano(p,x_data)
J

array([[ -1.      ,  -0.      ],
       [ -1.      ,  -1.052632],
       [ -1.      ,  -2.105263],
       [ -1.      ,  -3.157895],
       [ -1.      ,  -4.210526],
       [ -1.      ,  -5.263158],
       [ -1.      ,  -6.315789],
       [ -1.      ,  -7.368421],
       [ -1.      ,  -8.421053],
       [ -1.      ,  -9.473684],
       [ -1.      , -10.52632 ],
       [ -1.      , -11.57895 ],
       [ -1.      , -12.63158 ],
       [ -1.      , -13.68421 ],
       [ -1.      , -14.73684 ],
       [ -1.      , -15.78947 ],
       [ -1.      , -16.84211 ],
       [ -1.      , -17.89474 ],
       [ -1.      , -18.94737 ],
       [ -1.      , -20.      ]])

In [ ]:
J.T @ GetF(p)

array([ 357.5763586 , 4220.47358321])

In [ ]:
def Metric(p):
  return 0.5*np.linalg.norm(GetF(p))

In [ ]:
Metric(p)

np.float64(41.773228664181694)

In [ ]:
def Minimizer(p,lr=1e-3,epochs=int(1e5),error=1e-3):

  metric = 1
  it = 0

  M = np.array([])
  history = np.array([p])

  while metric > error and it < epochs:

    M = np.append(M,Metric(p))

    J = Jacobiano(p,x_data)
    Vector = GetF(p)

    # Algoritmo
    # Hagamos ML Levenberg–Marquardt
    lambda_ = 0.01
    NewJ = J.T @ J + lambda_*np.eye(len(p))
    #print(NewJ)

    grad = np.linalg.solve(NewJ,np.dot(J.T,Vector))
    #print(grad)
    p = p - lr*grad


    history = np.vstack((history,p))

    metric = Metric(p)

    if it % 100 == 0:
      clear_output(wait=True)
      _,ax,ax1,ax2 = Plotter(it)
      ax.plot(history[:it])
      ax.legend(['$u_{}$: {:.8}'.format(i, p[i]) for i in range(len(p))])
      ax1.plot(M[:it],color='k',label='Metric {:.3f}'.format(M[-1]))
      ax1.legend()

      ax2.scatter(x_data,y_data)
      _t = np.linspace(np.min(x_data),np.max(x_data),100)
      ax2.plot(_t,Model(p,_t),color='r',label='Model')
      ax2.legend()
      plt.show()

      time.sleep(0.05)

    it += 1


  return p

In [ ]:
p0 = np.array([-3.,1.,0, 0, 0,0,0])
xsol = Minimizer(p0)

KeyboardInterrupt: 

In [ ]:
xsol

array([1.77145834, 1.77344981])

In [ ]:
GetF(G1,xsol)

array([ 3.84497757e-07, -1.98948755e-03])